In [1]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
import sys, getopt

#%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    features, labels = np.empty((0,193)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            #mfccs, chroma, mel, contrast = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            #ext_features = np.hstack([mfccs,chroma,mel,contrast])
            features = np.vstack([features,ext_features])
            labels = np.append(labels, fn.split('\\')[2].split('-')[1])
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode


def load_wave_file(fname):
    print('Loading...', fname)
    features, labels = np.empty((0,193)), np.empty(0)
    mfccs, chroma, mel, contrast,tonnetz = extract_feature(fname)
    ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    features = np.vstack([features,ext_features])
    features = np.array(features)
    return features


training_epochs = 5000
#n_dim = features.shape[1]
n_dim = 193
n_classes = 10
#n_hidden_units_one = 280 
#n_hidden_units_two = 300
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.01


n_hidden_units_one = 300
n_hidden_units_two = 200
n_hidden_units_three = 100


X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,n_classes])

W_1 = tf.Variable(tf.random_normal([n_dim, n_hidden_units_one], mean=0, stddev=sd), name="w1")
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean=0, stddev=sd), name="b1")
h_1 = tf.nn.sigmoid(tf.matmul(X, W_1) + b_1)

W_2 = tf.Variable(tf.random_normal([n_hidden_units_one, n_hidden_units_two], mean=0, stddev=sd), name="w2")
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean=0, stddev=sd), name="b2")
h_2 = tf.nn.tanh(tf.matmul(h_1, W_2) + b_2)

W_3 = tf.Variable(tf.random_normal([n_hidden_units_two, n_hidden_units_three], mean=0, stddev=sd), name="w3")
b_3 = tf.Variable(tf.random_normal([n_hidden_units_three], mean=0, stddev=sd), name="b3")
h_3 = tf.nn.sigmoid(tf.matmul(h_2, W_3) + b_3)

W = tf.Variable(tf.random_normal([n_hidden_units_three, n_classes], mean=0, stddev=sd), name="w")
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd), name="b")
y_ = tf.nn.softmax(tf.matmul(h_3, W) + b)

cost_function = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y_), reduction_indices=[1]))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

# 정답값(Y)과 NN이 예측산출한 값(y_)이 일치하는지 검사
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))

# 전체 테스트 데이터 중 정답 == NN 산출값이 일치하는 비율.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 산출/예측한 값(y_)
predict_raw = y_
predict = tf.argmax(y_,1) #

y_true, y_pred = None, None


sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, "./sound_trained.ckpt")

for fname in glob.glob('audio_for_test/*.wav'):
    features =  load_wave_file(fname)
    raw, decision = sess.run([predict_raw, predict],feed_dict={X: features})
    #print("\tPredicted Raw", raw[0])
    print("\tPredicted Raw", ['%.5f' % r for r in raw[0]])
    print("\tDecision", decision[0])
    


INFO:tensorflow:Restoring parameters from ./sound_trained.ckpt
Loading... audio_for_test\145577-1-0-0.wav
	Predicted Raw ['0.00036', '0.44629', '0.00808', '0.54207', '0.00047', '0.00054', '0.00067', '0.00052', '0.00058', '0.00042']
	Decision 3
Loading... audio_for_test\204773-3-8-0.wav
	Predicted Raw ['0.00002', '0.00022', '0.00079', '0.99885', '0.00002', '0.00002', '0.00002', '0.00002', '0.00002', '0.00001']
	Decision 3
Loading... audio_for_test\60591-2-0-4.wav
	Predicted Raw ['0.00044', '0.02811', '0.10867', '0.86018', '0.00049', '0.00046', '0.00043', '0.00045', '0.00039', '0.00036']
	Decision 3
Loading... audio_for_test\60591-2-0-7.wav
	Predicted Raw ['0.00067', '0.06317', '0.76625', '0.16578', '0.00081', '0.00066', '0.00065', '0.00073', '0.00062', '0.00066']
	Decision 2
Loading... audio_for_test\7389-1-2-2.wav
	Predicted Raw ['0.00030', '0.90644', '0.08090', '0.01011', '0.00032', '0.00036', '0.00045', '0.00034', '0.00042', '0.00038']
	Decision 1
Loading... audio_for_test\74723-3-0-